# 搞清楚fine-tuning里面的代码

一些依赖

In [1]:
%matplotlib inline
import d2lzh as d2l
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata#, loss as gloss, model_zoo
# from mxnet.gluon import utils as gutils
import os

修改data位置,有数据集就不执行这一代码块了

In [2]:
data_dir = '../../data' ###
base_url = 'https://apache-mxnet.s3-accelerate.amazonaws.com/'
fname = gutils.download(
    base_url + 'gluon/dataset/hotdog.zip',
    path=data_dir, sha1_hash='fba480ffa8aa7e0febbb511d181409f899b9baa5')
with zipfile.ZipFile(fname, 'r') as z:
    z.extractall(data_dir)

In [3]:
train_imgs = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, 'hotdog/train'))
test_imgs = gdata.vision.ImageFolderDataset(
    os.path.join(data_dir, 'hotdog/test'))

`ImageFolderDataset`是一个类，继承于`dataset.Dataset`类

In [6]:
train_imgs

In [4]:
# 指定RGB三个通道的均值和方差来将图像通道归一化
normalize = gdata.vision.transforms.Normalize(
    [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])

train_augs = gdata.vision.transforms.Compose([
    gdata.vision.transforms.RandomResizedCrop(224),
    gdata.vision.transforms.RandomFlipLeftRight(),
    gdata.vision.transforms.ToTensor(),
    normalize])

test_augs = gdata.vision.transforms.Compose([
    gdata.vision.transforms.Resize(256),
    gdata.vision.transforms.CenterCrop(224),
    gdata.vision.transforms.ToTensor(),
    normalize])

训练数据所需要的迭代器，train_iter就是一个`DataLoader`类

In [5]:
batch_size = 128
train_iter = gdata.DataLoader(
        train_imgs.transform_first(train_augs), batch_size, shuffle=True)

`train_iter`就是一个含有上面传入数据集的迭代器，里面是一个一个的batch（列表类型），包含数据和标签。其实啊，还是上面那个nb的`ImageFolderDataset`适合这样标签就是文件夹名字的数据集

In [18]:
import mxnet as mx

for i, batch in enumerate(train_iter):
    if(i==6):
        # 打印一个batch看看
        batch6 = batch

features, labels = batch6 # 看看
ctx = mx.cpu(0)

# 要把ctx转化成一个列表，不然求不出长度
if isinstance(ctx, mx.Context):
        ctx = [ctx]

def get_batch(batch, ctx):
     """Return features and labels on ctx."""
     features, labels = batch   #labels是从这里来的
     if labels.dtype != features.dtype:
         labels = labels.astype(features.dtype)
         return (gutils.split_and_load(features, ctx),
                  gutils.split_and_load(labels, ctx), features.shape[0])

Xs, ys, batch_size = get_batch(batch6,ctx) # 在看看这个标签
print(Xs, ys, batch_size)

[
[[[[ 0.7590547   0.7590547   0.7761795  ...  0.2281874   0.15968838
     0.14256364]
   [ 0.7590547   0.7590547   0.7761795  ...  0.2281874   0.15968838
     0.14256364]
   [ 0.72480524  0.74193     0.7761795  ...  0.2624369   0.21106265
     0.1939379 ]
   ...
   [-1.5870366  -1.5699118  -1.5699118  ...  1.3412963   1.3412963
     1.3412963 ]
   [-1.5870366  -1.5870366  -1.5699118  ...  1.2899221   1.2727973
     1.2727973 ]
   [-1.5870366  -1.5870366  -1.5699118  ...  1.2899221   1.2727973
     1.2727973 ]]

  [[ 0.3452382   0.3452382   0.3627452  ... -0.24999997 -0.30252096
    -0.30252096]
   [ 0.3452382   0.3452382   0.3627452  ... -0.24999997 -0.30252096
    -0.30252096]
   [ 0.3102242   0.3102242   0.3452382  ... -0.21498597 -0.24999997
    -0.24999997]
   ...
   [-1.6680672  -1.6680672  -1.6330532  ...  0.29271722  0.2752102
     0.2577032 ]
   [-1.6680672  -1.6680672  -1.6505601  ...  0.2752102   0.2051822
     0.2051822 ]
   [-1.6680672  -1.6680672  -1.6505601  ...  0.27521

In [22]:
# 看看batch是什么东西
print(batch6)
print(type(batch6)) #就是一个list


[
[[[[ 0.7590547   0.7590547   0.7761795  ...  0.2281874   0.15968838
     0.14256364]
   [ 0.7590547   0.7590547   0.7761795  ...  0.2281874   0.15968838
     0.14256364]
   [ 0.72480524  0.74193     0.7761795  ...  0.2624369   0.21106265
     0.1939379 ]
   ...
   [-1.5870366  -1.5699118  -1.5699118  ...  1.3412963   1.3412963
     1.3412963 ]
   [-1.5870366  -1.5870366  -1.5699118  ...  1.2899221   1.2727973
     1.2727973 ]
   [-1.5870366  -1.5870366  -1.5699118  ...  1.2899221   1.2727973
     1.2727973 ]]

  [[ 0.3452382   0.3452382   0.3627452  ... -0.24999997 -0.30252096
    -0.30252096]
   [ 0.3452382   0.3452382   0.3627452  ... -0.24999997 -0.30252096
    -0.30252096]
   [ 0.3102242   0.3102242   0.3452382  ... -0.21498597 -0.24999997
    -0.24999997]
   ...
   [-1.6680672  -1.6680672  -1.6330532  ...  0.29271722  0.2752102
     0.2577032 ]
   [-1.6680672  -1.6680672  -1.6505601  ...  0.2752102   0.2051822
     0.2051822 ]
   [-1.6680672  -1.6680672  -1.6505601  ...  0.27521

In [10]:
train_iter

`batch_size`设置的128，一共2000个样本，故一共有15.625个批次，这里取16

In [16]:
train_iter.__len__()

16

在`d2l.train()`函数里，有一句是`for i, batch in enumerate(train_iter)`，就是从这个迭代器里取出一批一批的数据，下面是一些d2l里的代码

In [17]:
# 看看resnet152多大，230mb
pretrained_net = model_zoo.vision.resnet152_v2(pretrained=True)

In [18]:
pretrained_net

ResNetV2(
  (features): HybridSequential(
    (0): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=True, use_global_stats=False, in_channels=3)
    (1): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
    (3): Activation(relu)
    (4): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (5): HybridSequential(
      (0): BottleneckV2(
        (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv1): Conv2D(64 -> 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=64)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn3): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_st

In [19]:
pretrained_net.output

Dense(2048 -> 1000, linear)

In [ ]:
import os